In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_height, img_width = 64, 64
batch_size = 32

datagen = ImageDataGenerator(rescale=1.0/255)

train_data = datagen.flow_from_directory(
    'tai_chi/',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tensorflow.keras import layers, models

def build_generator(z_dim):
    model = models.Sequential()
    model.add(layers.Dense(256, input_dim=z_dim))
    model.add(layers.LeakyReLU(alpha=0.5))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.5))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.5))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(img_height * img_width * 3, activation='tanh'))
    model.add(layers.Reshape((img_height, img_width, 3)))
    return model

z_dim = 100  # Size of the noise vector
generator = build_generator(z_dim)

In [ ]:
def build_discriminator(img_shape):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.4))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.4))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

discriminator = build_discriminator((img_height, img_width, 3))
discriminator.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

In [ ]:
def build_gan(generator, discriminator):
    model = models.Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

discriminator.trainable = True
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def save_images(generator, epoch, z_dim, examples=32):
    noise = np.random.normal(0, 1, size=(examples, z_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale from [-1, 1] to [0, 1]

    # Create the directory if it doesn't exist
    directory = f'fin/_epoch_{epoch}/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Save each image separately
    for ind, img in enumerate(generated_images):
        plt.figure()
        plt.imshow(img, interpolation='nearest')
        plt.axis('off')
        plt.savefig(f'{directory}_{epoch}_image_{ind}.png')
        plt.close()

In [ ]:
import numpy as np
losses = []
def train_gan(generator, discriminator, gan, dataset, z_dim, epochs=100):
    for epoch in range(1, epochs+1):
        # Train Discriminator with real images
        if epoch % 2 == 0:
            real = dataset.next()
            real = real * 2.0 - 1.0
            real_labels = np.ones((real.shape[0], 1))*0.65
            d_loss_real = discriminator.train_on_batch(real, real_labels)

        # Train Discriminator with fake images
        # if epoch % 2 == 0:
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        fake = generator.predict(noise)
        fake_labels = np.zeros((batch_size, 1))+0.35
        d_loss_fake = discriminator.train_on_batch(fake, fake_labels)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)
        # Print progress
        if epoch % 10 == 0:
            losses.append(f"Epoch: {epoch} D_loss_real: {d_loss_real[0]} D_loss_fake: {d_loss_fake[0]} G_loss: {g_loss}")
            #save_images(generator, epoch, z_dim)

train_gan(generator, discriminator, gan, train_data, z_dim)

In [ ]:
losses